# ToDo 

Most of work is done and only the following needs to be done

AutoUnit-**trainging step**  https://pytorch.org/tnt/stable/framework/auto_unit.html
- add login W&B


In [1]:
import os
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import GPT2Tokenizer
from torch.utils.data import DataLoader
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.optim as optim
from torchtnt.framework.auto_unit import AutoUnit
from torchtnt.framework.callback import Callback
from torchtnt.framework.fit import fit

/home/remote/u1138167/BabyJoey/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# File paths to save/load the datasets
train_file = 'training_dataset.pt'
valid_file = 'validation_dataset.pt'

# Data and model configuration
data = "SouthernCrossAI/Project_Gutenberg_Australia"
sequence_length = 512
batch_size = 32
vocab_size = 50257
n_embd = 512
n_head = 8
n_layer_decoder = 1

In [3]:

# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', clean_up_tokenization_spaces=True)
tokenizer.pad_token = tokenizer.eos_token

In [4]:
# Tokenization function
def tokenize_function(dataset):
    return tokenizer(
        dataset['Paragraph'], 
        truncation=True, 
        padding='max_length', 
        max_length=sequence_length,
        return_attention_mask=True
    )

In [5]:

# Load and tokenize datasets
if os.path.exists(train_file) and os.path.exists(valid_file):
    # Load the transformed datasets
    training_dataset = torch.load(train_file)
    validation_dataset = torch.load(valid_file)
    print("Loaded existing transformed datasets.")
else:
    # Load the raw dataset
    ds = load_dataset(data)
    dataset = ds['train'].train_test_split(test_size=0.2)
    
    # Tokenize and transform the datasets
    training_dataset = dataset['train'].map(tokenize_function, batched=True)
    validation_dataset = dataset['test'].map(tokenize_function, batched=True)
    
    # Set format for PyTorch
    training_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
    validation_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
    
    # Save for future use
    torch.save(training_dataset, train_file)
    torch.save(validation_dataset, valid_file)
    print("Transformed datasets created and saved.")        

Loaded existing transformed datasets.


/tmp/ipykernel_1030273/334519769.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  training_dataset = torch.load(train_file)
/tmp/ipykernel_1030273/334519769.py:5: FutureW

In [6]:
# DataLoader for training and validation datasets
training_dataloader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

In [7]:
# Model Definition
class Embeddings(nn.Module):
    def __init__(self):
        super(Embeddings, self).__init__()
        self.token_embedding = nn.Embedding(vocab_size, n_embd)
        self.position_embedding = nn.Embedding(sequence_length, n_embd)

    def forward(self, x):
        tokens = self.token_embedding(x)
        positions = torch.arange(x.size(1), device=x.device).unsqueeze(0).expand_as(x)
        positions = self.position_embedding(positions)
        return tokens + positions

In [8]:
class TransformerBlock(nn.Module):
    def __init__(self):
        super(TransformerBlock, self).__init__()
        self.attn = nn.MultiheadAttention(n_embd, n_head)
        self.ln1 = nn.LayerNorm(n_embd)
        self.mlp = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd)
        )
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x, key_padding_mask=None):
        # Transpose for MultiheadAttention (seq_len, batch_size, embed_dim)
        x = x.transpose(0, 1)
        
        # Generate causal mask (triangular lower matrix to prevent attending to future tokens)
        seq_len = x.size(0)  # Sequence length after transpose
        attn_mask = torch.tril(torch.ones((seq_len, seq_len), device=x.device)).bool()  # Convert to boolean

        # Expand the causal mask to match the batch size and attention heads
        attn_mask = attn_mask.unsqueeze(0).expand(x.size(1) * n_head, -1, -1)

        # Pass the mask to the attention layer
        attn_output, _ = self.attn(x, x, x, attn_mask=attn_mask, key_padding_mask=key_padding_mask)
        
        x = x + attn_output
        x = self.ln1(x)

        # Transpose back to (batch_size, seq_len, embed_dim)
        x = x.transpose(0, 1)
        mlp_output = self.mlp(x)
        x = x + mlp_output
        x = self.ln2(x)

        return x


In [9]:
class BabyJoey(nn.Module):
    def __init__(self):
        super(BabyJoey, self).__init__()
        
        # Embeddings
        self.embeddings = Embeddings()
        
        # Decoder Blocks (based on n_layer_decoder)
        self.decoder_blocks = nn.ModuleList([TransformerBlock() for _ in range(n_layer_decoder)])

        # Output layer
        self.ln_f = nn.LayerNorm(n_embd)
        self.head = nn.Linear(n_embd, vocab_size, bias=False)

    def forward(self, x, attn_mask=None, key_padding_mask=None):
        # Get embeddings
        x = self.embeddings(x)

        # Apply decoder blocks with attention mask
        for block in self.decoder_blocks:
            x = block(x, key_padding_mask=key_padding_mask)  # Pass key_padding_mask to the block

        # Layer norm and output
        x = self.ln_f(x)
        logits = self.head(x)

        return logits

In [10]:
class BabyJoeyUnit(AutoUnit):
    def __init__(self, module, device=None):
        super().__init__(module=module, device=device)
        self.loss_fn = nn.CrossEntropyLoss()
    
    def compute_loss(self, state, data):
        input_ids, attention_mask = data['input_ids'], data['attention_mask']
        
        key_padding_mask = (attention_mask == 0).bool()
        logits = self.module(input_ids, key_padding_mask=key_padding_mask)

        # Shift input ids by one to get the target sequence
        targets = input_ids[:, 1:].contiguous()
        logits = logits[:, :-1, :].contiguous()
        
        loss = self.loss_fn(logits.view(-1, logits.size(-1)), targets.view(-1))
        return loss, logits

    def configure_optimizers_and_lr_scheduler(self, module):
        # Define the optimizer
        optimizer = optim.AdamW(module.parameters(), lr=1e-5, weight_decay=1e-3)  # Smaller learning rate and weight decay

        # Implement a learning rate scheduler
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
        
        return optimizer, scheduler


In [11]:
# Callback for training
class TestingCallback(Callback):
    def on_train_start(self, state, unit):
        print("Training started!")
    def on_train_end(self, state, unit):
        print("Training ended!")
    def on_train_epoch_start(self, state, unit):
        print(f"Training epoch {unit.train_progress.num_epochs_completed} started!")
    def on_train_epoch_end(self, state, unit):
        print(f"Training epoch {unit.train_progress.num_epochs_completed} ended!")
    def on_train_step_start(self, state, unit):
        print(f"Training batch {unit.train_progress.num_steps_completed} started!")
    def on_train_step_end(self, state, unit):
        print(f"Training batch {unit.train_progress.num_steps_completed} ended!")
    def on_eval_start(self, state, unit):
        print("Evaluation started!")
    def on_eval_end(self, state, unit):
        print("Evaluation ended!")
    def on_eval_epoch_start(self, state, unit):
        print(f"Evaluation epoch {unit.eval_progress.num_epochs_completed} started!")
    def on_eval_epoch_end(self, state, unit):
        print(f"Evaluation epoch {unit.eval_progress.num_epochs_completed} ended!")
    def on_eval_step_start(self, state, unit):
        print(f"Evaluation batch {unit.eval_progress.num_steps_completed} started!")
    def on_eval_step_end(self, state, unit):
        print(f"Evaluation batch {unit.eval_progress.num_steps_completed} ended!")
    def on_exception(self, state, unit, exc: BaseException):
        print(f"Exception occurred: {exc}")

In [12]:
# Define device and initialize the BabyJoey model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BabyJoey().to(device)
baby_joey_unit = BabyJoeyUnit(module=model, device=device)


In [13]:
for batch in training_dataloader:
    print(batch['input_ids'].shape)
    print(batch['attention_mask'].shape)
    break


torch.Size([32, 512])
torch.Size([32, 512])


In [14]:

# Train the model using the defined AutoUnit and callback
fit(
    baby_joey_unit,
    train_dataloader=training_dataloader,
    eval_dataloader=validation_dataloader,
    max_epochs=2,
    callbacks=[TestingCallback()]
)

Training started!
Training epoch 0 started!
Training batch 0 started!
Training batch 1 ended!
Training batch 1 started!
Training batch 2 ended!
Training batch 2 started!
Training batch 3 ended!
Training batch 3 started!
Training batch 4 ended!
Training batch 4 started!
Training batch 5 ended!
Training batch 5 started!
Training batch 6 ended!
Training batch 6 started!
Training batch 7 ended!
Training batch 7 started!
Training batch 8 ended!
Training batch 8 started!
Training batch 9 ended!
Training batch 9 started!
Training batch 10 ended!
Training batch 10 started!
Training batch 11 ended!
Training batch 11 started!
Training batch 12 ended!
Training batch 12 started!
Training batch 13 ended!
Training batch 13 started!
Training batch 14 ended!
Training batch 14 started!
Training batch 15 ended!
Training batch 15 started!
Training batch 16 ended!
Training batch 16 started!
Training batch 17 ended!
Training batch 17 started!
Training batch 18 ended!
Training batch 18 started!
Training bat